In [3]:
import pandas as pd
import glob
import datetime
import demoparser2 as d

pd.set_option('display.max_columns', 45)

pd.set_option('display.max_rows', 45)

parser = d.DemoParser("/home/felp/csdemos/dados/vitality-vs-natus-vincere-m1-nuke.dem")


# dim campeonato

In [4]:
campeonato = 'iem cologne'
tipo = 'bo5'
bracket = 'final'
data=datetime.datetime.now()
team1 = parser.parse_ticks(['team_clan_name'], ticks=[3000])['team_clan_name'].unique()[0]
team2 = parser.parse_ticks(['team_clan_name'], ticks=[3000])['team_clan_name'].unique()[1]
df_campeonato = pd.DataFrame([[campeonato, bracket, data, tipo, team1, team2, '', '']],
                             columns=['campeonato', 'stage', 'data', 'tipo', 'Time1', 'Time2', 'Vencedor', 'Placar'])

df_campeonato

,campeonato,stage,data,tipo,Time1,Time2,Vencedor,Placar
0,iem cologne,final,2024-08-20 00:09:02.095648,bo5,Team Vitality,Natus Vincere,,


# dim jogadores

In [5]:
df_jogadores = parser.parse_ticks(['team_clan_name'], ticks=[3000]).groupby(['team_clan_name', 'steamid', 'name']).count().reset_index().drop(columns=['tick'])

In [6]:
df_jogadores
#chave é steamdid

,team_clan_name,steamid,name
0,Natus Vincere,76561198013243326,Aleksib
1,Natus Vincere,76561198050250233,iM
2,Natus Vincere,76561198176878303,jL
3,Natus Vincere,76561198246607476,b1t
4,Natus Vincere,76561199063068840,w0nderful
5,Team Vitality,76561197973140692,mezii
6,Team Vitality,76561197978835160,flameZ
7,Team Vitality,76561197989744167,apEX
8,Team Vitality,76561198063336407,Spinx
9,Team Vitality,76561198113666193,ZywOo


# fato rounds

In [8]:
df_mapa_rounds = parser.parse_event("round_end").drop(columns=['round'])
mapa = parser.parse_header()['map_name']
df_mapa_rounds['mapa'] = mapa
df_mapa_rounds['round'] = [1+i for i in range(df_mapa_rounds.shape[0])]
df_mapa_rounds

,reason,tick,winner,mapa,round
0,bomb_defused,6310,CT,de_nuke,1
1,t_killed,12365,CT,de_nuke,2
2,t_killed,20322,CT,de_nuke,3
3,ct_killed,28437,T,de_nuke,4
4,ct_killed,33601,T,de_nuke,5
5,ct_killed,40593,T,de_nuke,6
6,t_killed,46807,CT,de_nuke,7
7,t_killed,58393,CT,de_nuke,8
8,bomb_defused,66727,CT,de_nuke,9
9,ct_killed,72795,T,de_nuke,10


In [24]:
wanted_fields = ['team_clan_name','team_name', 'kills_total', "deaths_total", "mvps", "headshot_kills_total",
                 "ace_rounds_total", "4k_rounds_total", "3k_rounds_total", 'damage_total', 'inventory', 'cash_spent_this_round', 'damage_total',
                 'utility_damage_total']
df_final = pd.DataFrame(columns=['inventory', 'cash_spent_this_round', 'kills_total', 'deaths_total',
                                    'headshot_kills_total', 'damage_total', 'utility_damage_total',
                                    'ace_rounds_total', '4k_rounds_total', '3k_rounds_total', 'mvps',
                                    'team_name', 'team_clan_name', 'tick', 'steamid', 'name', 'round'])
for i,j in enumerate(df_mapa_rounds['tick'].values):
    df = parser.parse_ticks(wanted_fields, ticks=[j])
    df['round'] = 1+i
    print(i)
    df_final = pd.concat([df_final, df])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [25]:
df_final

,inventory,cash_spent_this_round,kills_total,deaths_total,headshot_kills_total,damage_total,utility_damage_total,ace_rounds_total,4k_rounds_total,3k_rounds_total,mvps,team_name,team_clan_name,tick,steamid,name,round
0,[],650,0,1,0,0,0,0,0,0,0,CT,Team Vitality,6310,76561198063336407,Spinx,1
1,[],800,1,1,1,0,0,0,0,0,0,TERRORIST,Natus Vincere,6310,76561199063068840,w0nderful,1
2,[],650,2,1,2,0,0,0,0,0,0,TERRORIST,Natus Vincere,6310,76561198176878303,jL,1
3,"[knife_m9_bayonet, Dual Berettas]",650,1,0,1,0,0,0,0,0,0,CT,Team Vitality,6310,76561197989744167,apEX,1
4,[knife_m9_bayonet],800,1,0,0,0,0,0,0,0,0,CT,Team Vitality,6310,76561197978835160,flameZ,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,"[knife_karambit, High Explosive Grenade, Flash...",6200,21,20,16,2337,62,0,0,1,4,CT,Team Vitality,234050,76561197973140692,mezii,30
6,[],4800,28,24,17,3173,25,0,1,1,6,TERRORIST,Natus Vincere,234050,76561198050250233,iM,30
7,[],4900,12,23,9,1581,31,0,0,1,2,TERRORIST,Natus Vincere,234050,76561198246607476,b1t,30
8,[],6300,21,22,7,2448,22,0,0,1,3,CT,Team Vitality,234050,76561198113666193,ZywOo,30


In [14]:
pd.DataFrame(columns=['inventory', 'kills_total', 'deaths_total', 'headshot_kills_total',
       'damage_total', 'ace_rounds_total', '4k_rounds_total',
       '3k_rounds_total', 'mvps', 'team_name', 'team_clan_name', 'tick',
       'steamid', 'name', 'round'])

,inventory,kills_total,deaths_total,headshot_kills_total,damage_total,ace_rounds_total,4k_rounds_total,3k_rounds_total,mvps,team_name,team_clan_name,tick,steamid,name,round


In [13]:
df.columns

Index(['inventory', 'kills_total', 'deaths_total', 'headshot_kills_total',
       'damage_total', 'ace_rounds_total', '4k_rounds_total',
       '3k_rounds_total', 'mvps', 'team_name', 'team_clan_name', 'tick',
       'steamid', 'name', 'round'],
      dtype='object')

In [ ]:
df = parser.parse_event("player_death", player=["last_place_name", "team_name"], other=["total_rounds_played", "is_warmup_period"])

# filter out team-kills and warmup
df = df[df["attacker_team_name"] != df["user_team_name"]]
df = df[df["is_warmup_period"] == False]

# group-by like in sql
df = df.groupby(["total_rounds_played", "attacker_name"]).agg({'attacker_last_place_name':'count'}).unstack(0).reset_index().fillna(0)

In [ ]:
df.columns = ['round_'+str(1+int(j)) if j!='' else 'Player' for i,j in df.columns]

In [ ]:
df

In [ ]:
ticks_df = parser.parse_ticks(["X", "Y", 'health', 'score', 'inventory', 'rank', 'is_alive', 'death_time', 'game_time', 'cash_spent_this_round',
                               'active_weapon_name', 'time_last_injury', 'player_state','alive_time_total', 'round_start_time', 'round_win_status', 'round_win_reason', 
                               'total_rounds_played', 'player_death'])

ticks_df

In [ ]:
1+1

In [ ]:
ticks_df['rank'].value_counts()

In [ ]:
parser.parse_header()

# Players

In [ ]:
parser.parse_player_info()

In [ ]:
parser.parse_events(["all"])[0]